In [1]:
%cd ..

/home/zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg

import os.path as op
import time


from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras_tqdm import TQDMNotebookCallback

from cascading import cascade_net
from data import MaskedUntouched2DSequence

Using TensorFlow backend.


In [3]:
# paths
train_path = '../singlecoil_val/'
val_path = '../singlecoil_val/'

n_samples_train = 7135
n_samples_val = 7135

n_volumes_train = 199
n_volumes_val = 199

In [4]:
# generators
AF = 4
# MaskShifted2DSequence, MaskShiftedSingleImage2DSequence, MaskedUntouched2DSequence
train_gen = MaskedUntouched2DSequence(train_path, af=AF, inner_slices=8)
val_gen = MaskedUntouched2DSequence(val_path, af=AF)

In [5]:
run_params = {
    'n_cascade': 2, 
    'n_convs': 2, 
    'n_filters': 8,
}

n_epochs = 10
run_id = f'cascadenet_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'

In [6]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=2, save_weights_only=True)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
lr_on_plat_cback = ReduceLROnPlateau(monitor='val_loss', min_lr=5*1e-5, mode='auto', patience=3)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")

In [7]:
model = cascade_net(lr=1e-3, **run_params)
print(model.summary(line_length=100))

W0823 13:27:26.807692 139926072043264 deprecation_wrapper.py:119] From /home/zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0823 13:27:26.814299 139926072043264 deprecation.py:323] From /home/zaccharie/workspace/fastmri-reproducible-benchmark/pdnet_crop.py:59: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0823 13:27:26.841716 139926072043264 deprecation_wrapper.py:119] From /home/zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0823 13:27:26.847037 139926072043264 deprecation_wrapper.py:119] From /home/zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:4467: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W082

Model: "model_1"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
kspace_input (InputLayer)                        (None, 640, None, 1)             0                                                                   
______________________________________________________________________________________________________________________________________________________
mask_input (InputLayer)                          (None, 640, None)                0                                                                   
______________________________________________________________________________________________________________________________________________________
ifft_simple (Lambda)                             (None, 640, None, 1)        

In [ ]:
# simple overfit trials

data = train_gen[0]
val_data = val_gen[1]
model.fit(
    x=data[0], 
    y=data[1], 
    validation_data=val_data, 
    batch_size=data[0][0].shape[0], 
    epochs=100,
    verbose=2, 
    shuffle=False,
)

W0823 13:27:27.695096 139926072043264 deprecation_wrapper.py:119] From /home/zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:1021: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0823 13:27:27.752016 139926072043264 deprecation_wrapper.py:119] From /home/zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:1008: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 8 samples, validate on 38 samples
Epoch 1/100
